In [0]:
import requests
import pandas as pd

api_url = "https://fakestoreapi.com/products"
response = requests.get(api_url)
if response.ok:
    data = response.json()
    api_df = pd.DataFrame(data)
else:
    print("Request failed due to network restrictions in CE. Using uploaded table as a stand-in.")
    # Fallback: Read from uploaded source
    crm_df = spark.table("enterprise_modernization.default.crm_data_1")
    crm_df.show(5)


from pyspark.sql.functions import col

# Replace spaces and other invalid chars with underscores for all columns
def clean_colname(name):
    return name.strip().replace(' ', '_').replace('-', '_').replace('/', '_').replace('.', '_')

new_columns = [clean_colname(c) for c in crm_df.columns]   

df_clean = crm_df.toDF(*new_columns)

df_clean.write.mode("overwrite").saveAsTable("enterprise_modernization.bronze.bronze_crm_1")

In [0]:
# Read Bronze layer
df = spark.table("enterprise_modernization.bronze.bronze_crm_1")

print(df.columns)

print(df.count())

df.show(5)